# 广义线性模型 Generalized Linear Models
---
本章主要讲述一些用于回归的方法，其中目标值 y 是输入变量 x 的线性组合。 数学概念表示为：如果$\hat{y}$是预测值，那么有：

$\hat{y}(w, x) = w^{(0)} + w^{(1)} x^{(1)} + ... + w^{(p)} x^{(p)}$

在整个模块中，我们定义向量 $w = (w_1,..., w_p)$ 作为`coef_`(coefficient, 回归系数)，定义 $w_0$ 作为 `intercept_`截距.
线性模型虽简单, 却有丰富的变化.考虑单调可微函数$g(\cdot)$
($g(\cdot)$连续且充分光滑), 令

$$y = g^{-1}(w^Tx+b)$$
这样得到的模型称为`广义线性模型`,函数$g(\cdot)$称为`联系函数`(link function). 对数线性回归(Logistic Regression)是$g(\cdot)=ln(\cdot)$时的特例

## 线性回归
---
给定数据集 $D = \{(x_1, y_1), (x_2, y_2), \cdots, (x_n, y_n)\}$, 其中$x_i=(x_i^{(1)}, x_i^{(2)}, \cdots, x_i^{(d)}), y_i \in \mathbb{R}$. 线性回归(linear regression)试图学得一个线性模型以尽可能准确地预测实值输出标记.
$$f(x_i) = wx_i, 使得f(x_i)\simeq y_i$$

对于离散属性, 若属性值之间存在`序`(order)关系, 可通过连续化将其转为连续值; 若属性值间不存在序关系,假定有k个属性值, 则通常将其转为k维向量(OneHotEncoder)

## 普通最小二乘法 Ordinary Least Squares
---
LinearRegression 拟合一个带有系数$w = (w^{(0)}, ...,w^{(p)})$的线性模型，使得数据集实际观测数据和预测数据（估计值）之间的残差平方和(对应了常用的欧几里得距离Euclidean distance)最小。其数学表达式为:
$$\underset{w}{min} {|| X w - y||_2}^2$$
求解w使$E_{w} = \sum_{i=1}^{p}(w^Tx_i-y_i)^2$(loss function)最小化的过程, 称为线性回归模型的最小二乘`参数估计`(parameter estimation). $E_{w}$是关于$w$的凸函数, 可以令其对$w$并令导数为0, 得到$w$的最优解的闭式(closed-form)解.

用矩阵形式可以写成$E_{w} = (y-Xw)^T(y-Xw)$, 对$w$求导得到:
$\frac {\partial E_w}{\partial w} = 2X^T(Xw-y)$, 令其为0得到$\hat w = (X^TX)^{-1}X^Ty$

需要对矩阵求逆，因此这个方程只在逆矩阵存在的时候适用，我们在程序代码中对此作出判断。 判断矩阵是否可逆的一个可选方案是:

判断矩阵的行列式是否为 0，若为0，矩阵就不存在逆矩阵，不为0的话，矩阵才存在逆矩阵。

[矩阵求导参考1](https://blog.csdn.net/daaikuaichuan/article/details/80620518)

[矩阵求导参考2](http://blog.csdn.net/nomadlx53/article/details/50849941)

# 机器学习实战 例子1
___
数据格式
```
x0       x1       y
1.000000	0.067732	3.176513
1.000000	0.427810	3.816464
1.000000	0.995731	4.550095
1.000000	0.738336	4.256571
1.000000	0.981083	4.560815
```

In [29]:
import numpy as np
from copy import deepcopy
np.set_printoptions(threshold=15)

In [40]:
data = np.loadtxt('data.txt')
data

array([[1.      , 0.067732, 3.176513],
       [1.      , 0.42781 , 3.816464],
       [1.      , 0.995731, 4.550095],
       ...,
       [1.      , 0.070095, 3.213817],
       [1.      , 0.52707 , 3.952681],
       [1.      , 0.116163, 3.129283]])

In [41]:
np.all(data[:, 0]==1.)

True

In [42]:
# 取X 和 y
X, y = data[:, [1]], data[:, [2]]
X  # 保持 X 为 n_sample * n_feature

array([[0.067732],
       [0.42781 ],
       [0.995731],
       ...,
       [0.070095],
       [0.52707 ],
       [0.116163]])

In [62]:
y1 = np.mat(y).ravel()
y1.flatten()

matrix([[3.176513, 3.816464, 4.550095, ..., 3.213817, 3.952681, 3.129283]])

In [44]:
np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

array([[1.      , 0.067732],
       [1.      , 0.42781 ],
       [1.      , 0.995731],
       ...,
       [1.      , 0.070095],
       [1.      , 0.52707 ],
       [1.      , 0.116163]])

In [66]:
class LinearRegression:
    def __init__(self):
        self.intercept_ = None
        self.coef_ = None

    @staticmethod
    def transform(X):
        ones = np.ones((X.shape[0], 1))
        return np.concatenate((ones, X), axis=1)
    
    @staticmethod
    def least_squares(X, y):
        A = X.T * X
        if np.linalg.det(A) == 0.0:
            print('...')
        print(A.I.shape)
        w = A.I * X.T * y
        return w
    
    def fit(self, X, y):
        X = self.transform(deepcopy(X))
        print(X.shape)
        X = np.mat(X)
        y = np.mat(y)
        weights = self.least_squares(X, y)  # n_feature * 1
        self.intercept_ = weights[0, 0]
        self.coef_ = np.array(weights[1:, 0]).ravel()

In [67]:
clf = LinearRegression()
clf

In [68]:
clf.fit(X, y)

(200, 2)
(2, 2)


In [69]:
clf.coef_, clf.intercept_

(array([1.69532264]), 3.0077432426975914)